# Running the pipeline in a notebook

In [1]:
# imports
import os
import sys
import pprint

# project path
target_dir = os.path.dirname(os.getcwd())
target_dir_fraud = target_dir + "/fraud"
sys.path.extend([target_dir, target_dir_fraud])

import tfx
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.proto import example_gen_pb2
from tfx.v1 import proto
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.wrappers_pb2 import BoolValue
import pandas as pd
from sklearn import metrics

from configs import PROJECT_DIR, OUTPUT_DIR, TENSORBOARD_LOG_DIR, TRAIN_NUM_STEPS, EVAL_NUM_STEPS, SLICE_BY, LABEL_KEY, EVAL_ACCURACY_THRESHOLD
from local_runner import DATA_PATH, SERVING_MODEL_DIR

pp = pprint.PrettyPrinter()

# context
context = InteractiveContext(pipeline_root=OUTPUT_DIR)

## Load data

In [2]:
# load the data
output_config = proto.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        proto.SplitConfig.Split(name='train', hash_buckets=3),
        proto.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))
example_gen = tfx.components.CsvExampleGen(input_base=DATA_PATH, output_config=output_config)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 161
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 287
        type_id: 14
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/CsvExampleGen/examples/161"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:910847,xor_checksum:1632657850,sum_checksum:1632657850"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [3]:
# let us examine the outputs:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/CsvExampleGen/examples/161


In [4]:
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-eval')
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "AMT_CREDIT"
    value {
      float_list {
        value: 1102171.5
      }
    }
  }
  feature {
    key: "AMT_INCOME_TOTAL"
    value {
      float_list {
        value: 337500.0
      }
    }
  }
  feature {
    key: "CNT_CHILDREN"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "CODE_GENDER"
    value {
      bytes_list {
        value: "F"
      }
    }
  }
  feature {
    key: "DAYS_BIRTH"
    value {
      int64_list {
        value: -19380
      }
    }
  }
  feature {
    key: "DAYS_EMPLOYED"
    value {
      int64_list {
        value: -2632
      }
    }
  }
  feature {
    key: "FLAG_MOBIL"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "FLAG_OWN_CAR"
    value {
      bytes_list {
        value: "N"
      }
    }
  }
  feature {
    key: "FLAG_OWN_REALTY"
    value {
      bytes_list {
        value: "Y"
      }
    }
  }
  feature {
    key: "NAME_CONTRACT_TYPE

2021-10-09 17:59:44.403828: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-09 17:59:44.437466: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


## Generate statistics

In [5]:
# statistics
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'], exclude_splits=None)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 162
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 288
        type_id: 16
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/StatisticsGen/statistics/162"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [6]:
context.show(statistics_gen.outputs['statistics'])

## Generate schema

In [7]:
# schema
schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 163
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 289
        type_id: 18
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/SchemaGen/schema/163"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [8]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'AMT_CREDIT',FLOAT,required,,-
'AMT_INCOME_TOTAL',FLOAT,required,,-
'CNT_CHILDREN',INT,required,,-
'CODE_GENDER',STRING,required,,'CODE_GENDER'
'DAYS_BIRTH',INT,required,,-
'DAYS_EMPLOYED',INT,required,,-
'FLAG_MOBIL',INT,required,,-
'FLAG_OWN_CAR',STRING,required,,'FLAG_OWN_CAR'
'FLAG_OWN_REALTY',STRING,required,,'FLAG_OWN_REALTY'


/Users/ivan/.pyenv/versions/3.8.12/envs/py-tfx-38/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'CODE_GENDER',"'F', 'M'"
'FLAG_OWN_CAR',"'N', 'Y'"
'FLAG_OWN_REALTY',"'N', 'Y'"
'NAME_CONTRACT_TYPE',"'Cash loans', 'Revolving loans'"
'NAME_TYPE_SUITE',"'Children', 'Family', 'Group of people', 'Other_A', 'Other_B', 'Spouse, partner', 'Unaccompanied'"
'OCCUPATION_TYPE',"'Accountants', 'Cleaning staff', 'Cooking staff', 'Core staff', 'Drivers', 'HR staff', 'High skill tech staff', 'IT staff', 'Laborers', 'Low-skill Laborers', 'Managers', 'Medicine staff', 'Private service staff', 'Realty agents', 'Sales staff', 'Secretaries', 'Security staff', 'Waiters/barmen staff'"
'ORGANIZATION_TYPE',"'Advertising', 'Agriculture', 'Bank', 'Business Entity Type 1', 'Business Entity Type 2', 'Business Entity Type 3', 'Cleaning', 'Construction', 'Culture', 'Electricity', 'Emergency', 'Government', 'Hotel', 'Housing', 'Industry: type 1', 'Industry: type 10', 'Industry: type 11', 'Industry: type 12', 'Industry: type 13', 'Industry: type 2', 'Industry: type 3', 'Industry: type 4', 'Industry: type 5', 'Industry: type 6', 'Industry: type 7', 'Industry: type 8', 'Industry: type 9', 'Insurance', 'Kindergarten', 'Legal Services', 'Medicine', 'Military', 'Mobile', 'Other', 'Police', 'Postal', 'Realtor', 'Religion', 'Restaurant', 'School', 'Security', 'Security Ministries', 'Self-employed', 'Services', 'Telecom', 'Trade: type 1', 'Trade: type 2', 'Trade: type 3', 'Trade: type 5', 'Trade: type 6', 'Trade: type 7', 'Transport: type 1', 'Transport: type 2', 'Transport: type 3', 'Transport: type 4', 'University', 'XNA', 'Trade: type 4'"


## Validate data

In [9]:
# example validation
example_validator = tfx.components.ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 164
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 290
        type_id: 20
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/ExampleValidator/anomalies/164"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [10]:
context.show(example_validator.outputs['anomalies'])

/Users/ivan/.pyenv/versions/3.8.12/envs/py-tfx-38/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Transform data

In [11]:
# transform using preprocessing_fn
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=PROJECT_DIR + '/fraud/preprocessing.py'
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying configs.py -> build/lib
copying features.py -> build/lib
copying local_runner.py -> build/lib
copying preprocessing.py -> build/lib
copying pipeline.py -> build/lib
installing to /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp1tig549x
running install
running install_lib
copying build/lib/configs.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp1tig549x
copying build/lib/features.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp1tig549x
copying build/lib/local_runner.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp1tig549x
copying build/lib/preprocessing.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp1tig549x
copying build/lib/pipeline.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmp1tig549x
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_

Instructions for updating:
Use ref() instead.


Processing /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/_wheels/tfx_user_code_Transform-0.0+0e3dfdfb611e359e140d1a33c2fd0511f579b8ada3eb7c1ae43f4d75c9b48898-py3-none-any.whl


2021-10-09 18:00:09.110811: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/165/.temp_path/tftransform_tmp/010a351593ad47a5a330b7c335479442/assets


INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/165/.temp_path/tftransform_tmp/010a351593ad47a5a330b7c335479442/assets


INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/165/.temp_path/tftransform_tmp/513eb481d68b49d093756e44c63b8487/assets


INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/165/.temp_path/tftransform_tmp/513eb481d68b49d093756e44c63b8487/assets


ExecutionResult(
    component_id: Transform
    execution_id: 165
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 291
        type_id: 22
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transform_graph/165"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 292
        type_id: 14
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/transformed_examples/165"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 293
        type_id: 23
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/updated_analyzer_cache/165"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 294
        type_id: 18
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Transform/pre_transform_schema/165"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published

In [12]:
# training
trainer_args = {
    'module_file': PROJECT_DIR + '/fraud/estimators/wide_and_deep.py',
    'examples': transform.outputs['transformed_examples'],
    'schema': schema_gen.outputs['schema'],
    'transform_graph': transform.outputs['transform_graph'],
    'train_args': tfx.v1.proto.TrainArgs(num_steps=TRAIN_NUM_STEPS),
    'eval_args': tfx.v1.proto.EvalArgs(num_steps=EVAL_NUM_STEPS),
}
trainer = tfx.components.Trainer(**trainer_args)
context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying wide_and_deep.py -> build/lib
installing to /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmpsb5snt4f
running install
running install_lib
copying build/lib/wide_and_deep.py -> /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmpsb5snt4f
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/folders/8_/pl9fgq297ld3b7kgy5tmvf700000gn/T/tmpsb5snt4f/tfx_user_code_Trainer-0.0+10f558de898548e03a8cbd74ef9c8ca697dd7ccfcbd992f1a487c25

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Epoch 1/20


2021-10-09 18:00:28.175711: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-10-09 18:00:28.175768: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-10-09 18:00:28.176690: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-10-09 18:00:28.307150: W tensorflow/core/framework/dataset.cc:679] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


...

50/50 [==============================] - 3s 61ms/step - loss: 0.2680 - binary_accuracy: 0.9199 - auc: 0.6572 - val_loss: 0.2636 - val_binary_accuracy: 0.9247 - val_auc: 0.6081
Epoch 10/20
50/50 [==============================] - ETA: 0s - loss: 0.2278 - binary_accuracy: 0.9360 - auc: 0.674 - ETA: 0s - loss: 0.2641 - binary_accuracy: 0.9214 - auc: 0.656 - ETA: 0s - loss: 0.2648 - binary_accuracy: 0.9210 - auc: 0.660 - ETA: 0s - loss: 0.2653 - binary_accuracy: 0.9208 - auc: 0.660 - ETA: 0s - loss: 0.2666 - binary_accuracy: 0.9202 - auc: 0.661 - ETA: 0s - loss: 0.2660 - binary_accuracy: 0.9204 - auc: 0.661 - ETA: 0s - loss: 0.2661 - binary_accuracy: 0.9204 - auc: 0.661 - 3s 59ms/step - loss: 0.2662 - binary_accuracy: 0.9203 - auc: 0.6623 - val_loss: 0.2634 - val_binary_accuracy: 0.9247 - val_auc: 0.6096
Epoch 11/20
50/50 [==============================] - ETA: 0s - loss: 0.2836 - binary_accuracy: 0.9120 - auc: 0.683 - ETA: 0s - loss: 0.2627 - binary_accuracy: 0.9217 - auc: 0.663 - ETA: 0s

2021-10-09 18:01:32.822346: W tensorflow/core/framework/dataset.cc:679] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


50/50 [==============================] - 4s 79ms/step - loss: 0.2643 - binary_accuracy: 0.9198 - auc: 0.6791 - val_loss: 0.2633 - val_binary_accuracy: 0.9247 - val_auc: 0.6154
Epoch 20/20
50/50 [==============================] - ETA: 0s - loss: 0.2744 - binary_accuracy: 0.9160 - auc: 0.668 - ETA: 0s - loss: 0.2636 - binary_accuracy: 0.9200 - auc: 0.678 - ETA: 0s - loss: 0.2640 - binary_accuracy: 0.9198 - auc: 0.679 - ETA: 0s - loss: 0.2627 - binary_accuracy: 0.9203 - auc: 0.680 - ETA: 0s - loss: 0.2630 - binary_accuracy: 0.9202 - auc: 0.680 - ETA: 0s - loss: 0.2641 - binary_accuracy: 0.9197 - auc: 0.680 - ETA: 0s - loss: 0.2628 - binary_accuracy: 0.9203 - auc: 0.680 - 4s 81ms/step - loss: 0.2631 - binary_accuracy: 0.9202 - auc: 0.6805 - val_loss: 0.2633 - val_binary_accuracy: 0.9248 - val_auc: 0.6146
INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model/166/Format-Serving/assets


INFO:tensorflow:Assets written to: /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model/166/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 166
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 299
        type_id: 25
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model/166"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 300
        type_id: 26
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model_run/166"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
%reload_ext tensorboard
%tensorboard --logdir {TENSORBOARD_LOG_DIR}

## Evaluation

In [14]:
# first lets add a model resolver
model_resolver = tfx.v1.dsl.Resolver(
    strategy_class=tfx.v1.dsl.experimental.LatestBlessedModelStrategy,
    model=tfx.v1.dsl.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessing=tfx.v1.dsl.Channel(type=tfx.types.standard_artifacts.ModelBlessing)
).with_id('latest_blessed_model_resolver')
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 167
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
eval_config = tfma.EvalConfig(
    options=tfma.Options(include_default_metrics=BoolValue(value=True)),
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=f'{LABEL_KEY}_xf',
            preprocessing_function_names=['transform_features'])
    ],
    slicing_specs=[tfma.SlicingSpec()] + [tfma.SlicingSpec(feature_keys=[spec]) for spec in ['CODE_GENDER']],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(
                class_name='AUC',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': EVAL_ACCURACY_THRESHOLD}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': -1e-10})))
        ])
    ])
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)
context.run(evaluator)

ERROR:absl:There are change thresholds, but the baseline is missing. This is allowed only when rubber stamping (first run).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x175af9220> and <keras.engine.input_layer.InputLayer object at 0x1736bc040>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x175af9220> and <keras.engine.input_layer.InputLayer object at 0x1736bc040>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1756d1d90> and <keras.engine.input_layer.InputLayer object at 0x1752112b0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x1756d1d90> and <keras.engine.input_layer.InputLayer object at 0x1752112b0>).


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x174618640> and <keras.engine.input_layer.InputLayer object at 0x173a913a0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x174618640> and <keras.engine.input_layer.InputLayer object at 0x173a913a0>).


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 168
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 301
        type_id: 29
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Evaluator/evaluation/168"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 29
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 302
        type_id: 30
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Evaluator/blessing/168"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model/166"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 299
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 30
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [16]:
context.show(evaluator.outputs['evaluation'])

In [17]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)
tfma.view.render_slicing_metrics(tfma_result, slicing_column='CODE_GENDER')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'CODE_GENDER:F', 'met…

In [18]:
blessing_loc = evaluator.outputs['blessing'].get()[0].uri
! ls {blessing_loc}

BLESSED


## Pushing the model

In [19]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.v1.proto.PushDestination(filesystem=tfx.v1.proto.PushDestination.Filesystem(base_directory=SERVING_MODEL_DIR))
)
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 169
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 303
        type_id: 32
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Pusher/pushed_model/169"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/tfx_pipeline_output/fraud/serving_model/1633795352"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1633795352"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [20]:
pusher_args = {'model': trainer.outputs['model'], 'model_blessing': evaluator.outputs['blessing'],
               'push_destination': tfx.v1.proto.PushDestination(
                   filesystem=tfx.v1.proto.PushDestination.Filesystem(base_directory=SERVING_MODEL_DIR)
               )}
pusher = tfx.components.Pusher(**pusher_args)
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 170
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 303
        type_id: 32
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Pusher/pushed_model/169"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/tfx_pipeline_output/fraud/serving_model/1633795352"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1633795352"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1633795352297
        last_update_time_since_epoch: 1633795352374
        , artifact_type: id: 32
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [21]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
if os.listdir(push_uri):
    model = tf.saved_model.load(push_uri)
    for item in model.signatures.items():
        pp.pprint(item)

('serving_default',
 <ConcreteFunction signature_wrapper(*, examples) at 0x17ADF1EB0>)
('transform_features',
 <ConcreteFunction signature_wrapper(*, examples) at 0x17AED3160>)


## Inference

In [22]:
TEST_DATA_PATH = '/Users/ivan/Documents/dev/learn/tfx/data/test_data'
infer_output_config = proto.Output(split_config=example_gen_pb2.SplitConfig(splits=[proto.SplitConfig.Split(name='test', hash_buckets=1)]))
new_example_gen = tfx.components.CsvExampleGen(input_base=TEST_DATA_PATH, output_config=infer_output_config)
context.run(new_example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 171
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 304
        type_id: 14
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/CsvExampleGen/examples/171"
        properties {
          key: "split_names"
          value {
            string_value: "[\"test\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:456393,xor_checksum:1632657850,sum_checksum:1632657850"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [23]:
inferrer = tfx.components.BulkInferrer(
    examples=new_example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    data_spec=tfx.proto.bulk_inferrer_pb2.DataSpec(),
    model_spec=tfx.proto.bulk_inferrer_pb2.ModelSpec(),
    output_example_spec=tfx.proto.bulk_inferrer_pb2.OutputExampleSpec(
         output_columns_spec=[tfx.proto.bulk_inferrer_pb2.OutputColumnsSpec(
             predict_output=tfx.proto.bulk_inferrer_pb2.PredictOutput(
                 output_columns=[tfx.proto.bulk_inferrer_pb2.PredictOutputCol(output_key='outputs',output_column='TARGET_PRED', )]
             )
         )]
     )
)
context.run(inferrer)

infer_statistics = tfx.components.StatisticsGen(examples=inferrer.outputs['output_examples'])
context.run(infer_statistics)

infer_schema = tfx.components.SchemaGen(statistics=infer_statistics.outputs['statistics'])
context.run(infer_schema)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


INFO:tensorflow:Restoring parameters from /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model/166/Format-Serving/variables/variables


INFO:tensorflow:Restoring parameters from /Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/Trainer/model/166/Format-Serving/variables/variables


ExecutionResult(
    component_id: SchemaGen
    execution_id: 174
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 307
        type_id: 18
        uri: "/Users/ivan/Documents/dev/learn/tfx/pipeline_outputs/SchemaGen/schema/174"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [24]:
def decode_fn(record_bytes):
    parsed = tf.io.parse_single_example(
        record_bytes,
        {"TARGET": tf.io.FixedLenFeature([], dtype=tf.int64),
         "TARGET_PRED": tf.io.FixedLenFeature([], dtype=tf.float32)})
    return parsed

pred_dict = {"TARGET": [], "TARGET_PRED": []}

prediction_logs_dir = inferrer.outputs['output_examples'].get()[0].uri + '/Split-test'
tfrecord_filenames = [os.path.join(prediction_logs_dir, name) for name in os.listdir(prediction_logs_dir)]
prediction_data = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for preds in prediction_data.map(decode_fn).batch(10):
    for k, v in preds.items():
        pred_dict[k].extend(v.numpy())

pred_df = pd.DataFrame.from_dict(pred_dict)
pred_df.head()

,TARGET,TARGET_PRED
0,0,0.020396
1,0,0.151595
2,0,0.084454
3,0,0.082158
4,0,0.067946


In [25]:
# calculate AUC
fpr, tpr, thresholds = metrics.roc_curve(pred_df['TARGET'].values, pred_df['TARGET_PRED'].values, pos_label=1)
metrics.auc(fpr, tpr)

0.6276305468890936